# Health Insurance Project: Data Import, Clean & Transform
---

Import Pandas library:

In [1]:
import pandas as pd
import numpy as np

---

## Import and check the data

Import the insurance_raw.csv file:

In [2]:
insurance_raw = pd.read_csv("../data/insurance_raw.csv")

View the first 10 rows of the raw file:

In [3]:
insurance_raw.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


Check summary of inital raw data:

In [4]:
before_summary = pd.DataFrame({
    "dtype": insurance_raw.dtypes,
    "missing_values": insurance_raw.isnull().sum(),
    "non_missing": insurance_raw.notnull().sum(),
    "unique_values": insurance_raw.nunique()
})
before_summary

,dtype,missing_values,non_missing,unique_values
age,int64,0,1338,47
sex,object,0,1338,2
bmi,float64,0,1338,548
children,int64,0,1338,6
smoker,object,0,1338,2
region,object,0,1338,4
charges,float64,0,1338,1337


There are no missing values, so no need to handle any missing data.

Age and bmi have very varied results, looking at the unique values, so I will look to add new bin/banding groups for both of them.

Sex, smoker and region have very few unique values so look like category columns. I will view the unique values and set them as category columns.

Children i will leave as an integer, but will add a new column to flag if it includes family plan.


---

## Rename columns

Make a copy of the DataFrame to work with:

In [5]:
insurance = insurance_raw.copy()

Rename all the column names to title case:

In [6]:
insurance.columns = [col.title() for col in insurance.columns]

Change the Bmi column name to be BMI all capitals:

In [7]:
insurance.rename(columns={'Bmi': 'BMI'}, inplace=True)

Display the updated column names:

In [8]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       1338 non-null   int64  
 1   Sex       1338 non-null   object 
 2   BMI       1338 non-null   float64
 3   Children  1338 non-null   int64  
 4   Smoker    1338 non-null   object 
 5   Region    1338 non-null   object 
 6   Charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


---

## Address categorical columns

View the unique values in each of the columns that will be categories:

In [9]:
categorical_columns = ['Sex', 'Smoker', 'Region']

for col in categorical_columns:
    print(f"Unique values in {col} are : {insurance[col].unique()}")

Unique values in Sex are : ['female' 'male']
Unique values in Smoker are : ['yes' 'no']
Unique values in Region are : ['southwest' 'southeast' 'northwest' 'northeast']


Set these to be category data type:

In [10]:
categorical_columns = ['Sex', 'Smoker', 'Region']

for col in categorical_columns:
    insurance[col] = insurance[col].astype('category')

And view the updated column types:

In [11]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Age       1338 non-null   int64   
 1   Sex       1338 non-null   category
 2   BMI       1338 non-null   float64 
 3   Children  1338 non-null   int64   
 4   Smoker    1338 non-null   category
 5   Region    1338 non-null   category
 6   Charges   1338 non-null   float64 
dtypes: category(3), float64(2), int64(2)
memory usage: 46.3 KB


---

## Add Age Groups

Googling US Health Insurance age brackets, there isn't a defined range that is in place.

After viewing [KFF Site](https://www.kff.org/affordable-care-act/state-indicator/marketplace-plan-selection-by-age/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D) and [State Health Compare Site](https://statehealthcompare.shadac.org/table/4/health-insurance-coverage-type-by-age#1/5,4,1,10,86,9,8,6,3,12,13,20,25,14,21,22,23,24,11/76/7,8), i have decided to go with these age groups as the age range is 18-64:

- `< 26` - 'Ages 18-25'
- `>= 26 < 35` - 'Ages 26-34'
- `>= 35 < 45` - 'Ages 35-44'
- `>= 44 < 55` - 'Ages 44-54'
- `>= 55` - 'Ages 55 and over'

In [12]:
insurance['Age_Group'] = pd.cut(
    insurance['Age'],
    bins=[17, 26, 35, 45, 55, 100],
    labels=['Ages 18-25', 'Ages 26-34', 'Ages 35-44', 'Ages 45-54', 'Ages 55 and over'],
    right=False
)

insurance.head()

,Age,Sex,BMI,Children,Smoker,Region,Charges,Age_Group
0,19,female,27.900,0,yes,southwest,16884.92400,Ages 18-25
1,18,male,33.770,1,no,southeast,1725.55230,Ages 18-25
2,28,male,33.000,3,no,southeast,4449.46200,Ages 26-34
3,33,male,22.705,0,no,northwest,21984.47061,Ages 26-34
4,32,male,28.880,0,no,northwest,3866.85520,Ages 26-34


---

## Add BMI Groups

On the [NHS Site](https://www.nhs.uk/conditions/obesity/) it defines the BMI groups to be:

- `below 18.5` – you're in the underweight range
- `18.5 to 24.9` – you're in the healthy weight range
- `25 to 29.9` – you're in the overweight range
- `30 to 39.9` – you're in the obese range
- `40 or above` – you're in the severely obese range

So will set my BMI_Group to be:

- `< 18.5` - 'Underweight'
- `>= 18.5 < 25` - 'Healthy Weight'
- `>= 25 < 30` - 'Overweight'
- `>= 30 < 40` - 'Obese'
- `>= 40` - 'Severely Obese'

In [13]:
insurance['BMI_Group'] = pd.cut(
    insurance['BMI'],
    bins=[0, 18.5, 25, 30, 40, 100],
    labels=['Underweight', 'Healthy Weight', 'Overweight', 'Obese', 'Severely Obese'],
    right=False
)

insurance.head()

,Age,Sex,BMI,Children,Smoker,Region,Charges,Age_Group,BMI_Group
0,19,female,27.900,0,yes,southwest,16884.92400,Ages 18-25,Overweight
1,18,male,33.770,1,no,southeast,1725.55230,Ages 18-25,Obese
2,28,male,33.000,3,no,southeast,4449.46200,Ages 26-34,Obese
3,33,male,22.705,0,no,northwest,21984.47061,Ages 26-34,Healthy Weight
4,32,male,28.880,0,no,northwest,3866.85520,Ages 26-34,Overweight


---

## Add Plan column

I will add a new Plan column that will be Family if Children is > 0 or Standard if its just individual with no children and make a category column

In [14]:
insurance['Plan'] = np.where(insurance['Children'] > 0, 'Family', 'Standard')
insurance['Plan'] = insurance['Plan'].astype('category')
insurance.head()

,Age,Sex,BMI,Children,Smoker,Region,Charges,Age_Group,BMI_Group,Plan
0,19,female,27.900,0,yes,southwest,16884.92400,Ages 18-25,Overweight,Standard
1,18,male,33.770,1,no,southeast,1725.55230,Ages 18-25,Obese,Family
2,28,male,33.000,3,no,southeast,4449.46200,Ages 26-34,Obese,Family
3,33,male,22.705,0,no,northwest,21984.47061,Ages 26-34,Healthy Weight,Standard
4,32,male,28.880,0,no,northwest,3866.85520,Ages 26-34,Overweight,Standard


---

## Title case existing category values

Change the category values for Sex, Smoker and Regio so all the values have a capital first letter

In [16]:
insurance['Sex'] = insurance['Sex'].cat.rename_categories(lambda x: x.title())
insurance['Smoker'] = insurance['Smoker'].cat.rename_categories(lambda x: x.title())
insurance['Region'] = insurance['Region'].cat.rename_categories(lambda x: x.title())
insurance.head()

,Age,Sex,BMI,Children,Smoker,Region,Charges,Age_Group,BMI_Group,Plan
0,19,Female,27.900,0,Yes,Southwest,16884.92400,Ages 18-25,Overweight,Standard
1,18,Male,33.770,1,No,Southeast,1725.55230,Ages 18-25,Obese,Family
2,28,Male,33.000,3,No,Southeast,4449.46200,Ages 26-34,Obese,Family
3,33,Male,22.705,0,No,Northwest,21984.47061,Ages 26-34,Healthy Weight,Standard
4,32,Male,28.880,0,No,Northwest,3866.85520,Ages 26-34,Overweight,Standard


---

## Add charges per person column

I am adding a column that is the charges divided by the number of people included (Children + 1) to get a cost per person, which i may or may not use in my investigation

In [17]:
insurance['Charges_Per_Person'] = insurance['Charges'] / (insurance['Children'] + 1)
insurance.head()

,Age,Sex,BMI,Children,Smoker,Region,Charges,Age_Group,BMI_Group,Plan,Charges_Per_Person
0,19,Female,27.900,0,Yes,Southwest,16884.92400,Ages 18-25,Overweight,Standard,16884.92400
1,18,Male,33.770,1,No,Southeast,1725.55230,Ages 18-25,Obese,Family,862.77615
2,28,Male,33.000,3,No,Southeast,4449.46200,Ages 26-34,Obese,Family,1112.36550
3,33,Male,22.705,0,No,Northwest,21984.47061,Ages 26-34,Healthy Weight,Standard,21984.47061
4,32,Male,28.880,0,No,Northwest,3866.85520,Ages 26-34,Overweight,Standard,3866.85520


---

## Add outlier columns

Create a function to see if a charge or charge_per_person is an outlier and falls outside 1.5 * IQR

Use the function to add a new column that sets true or false as its value depending if it is an outlier

In [19]:
def check_outlier(s):
    """ Calculating inter quartile range and returning True or False"""
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    return (s < low) | (s > high)

insurance['Charges_Outlier'] = check_outlier(insurance['Charges'])
insurance['Charges_Per_Person_Outlier'] = check_outlier(insurance['Charges_Per_Person'])
insurance.head()

,Age,Sex,BMI,Children,Smoker,Region,Charges,Age_Group,BMI_Group,Plan,Charges_Per_Person,Charges_Outlier,Charges_Per_Person_Outlier
0,19,Female,27.900,0,Yes,Southwest,16884.92400,Ages 18-25,Overweight,Standard,16884.92400,False,False
1,18,Male,33.770,1,No,Southeast,1725.55230,Ages 18-25,Obese,Family,862.77615,False,False
2,28,Male,33.000,3,No,Southeast,4449.46200,Ages 26-34,Obese,Family,1112.36550,False,False
3,33,Male,22.705,0,No,Northwest,21984.47061,Ages 26-34,Healthy Weight,Standard,21984.47061,False,False
4,32,Male,28.880,0,No,Northwest,3866.85520,Ages 26-34,Overweight,Standard,3866.85520,False,False


---

## Add numeric columns for categories

So that I can include the category columns in correlation checks, i will add a numeric equivalent column for each of the categories and set the value as the category code

In [21]:
cat_cols = ['Sex', 'Smoker', 'Region', 'Plan', 'BMI_Group', 'Age_Group']
for col in cat_cols:
    insurance[f"{col}_num"] = insurance[col].cat.codes

insurance.head()

,Age,Sex,BMI,Children,Smoker,Region,Charges,Age_Group,BMI_Group,Plan,Charges_Per_Person,Charges_Outlier,Charges_Per_Person_Outlier,Sex_num,Smoker_num,Region_num,Plan_num,BMI_Group_num,Age_Group_num
0,19,Female,27.900,0,Yes,Southwest,16884.92400,Ages 18-25,Overweight,Standard,16884.92400,False,False,0,1,3,1,2,0
1,18,Male,33.770,1,No,Southeast,1725.55230,Ages 18-25,Obese,Family,862.77615,False,False,1,0,2,0,3,0
2,28,Male,33.000,3,No,Southeast,4449.46200,Ages 26-34,Obese,Family,1112.36550,False,False,1,0,2,0,3,1
3,33,Male,22.705,0,No,Northwest,21984.47061,Ages 26-34,Healthy Weight,Standard,21984.47061,False,False,1,0,1,1,1,1
4,32,Male,28.880,0,No,Northwest,3866.85520,Ages 26-34,Overweight,Standard,3866.85520,False,False,1,0,1,1,2,1
